In [1]:
from Bio import AlignIO
import os

In [2]:
cadherin_msa_path = os.path.join('..', 'data', 'cadherin', 'PF00028.alignment.seed')
alignment = AlignIO.read(cadherin_msa_path, 'stockholm')

In [3]:
seqs = []

for record in alignment:
    seqs.append(str(record.seq).replace('-', ''))
    # seqs.append(record.seq)


In [4]:
seqs[0]

'QTVRIKENVPVGTKTIGYKAYDPETGSSSGIRYKKSSDPEGWVDVDKNSGVITILKRLDREARSGVYNISIIASDKDGRTCNGVLGIVLE'

In [5]:
seqs

['QTVRIKENVPVGTKTIGYKAYDPETGSSSGIRYKKSSDPEGWVDVDKNSGVITILKRLDREARSGVYNISIIASDKDGRTCNGVLGIVLE',
 'KVIQSEDCLPTGTELLGYKAVDPERGTGEGLRYKKIQDEDNWFEINEYTGDLKTVKVLDRESTFVKNNQYNVSVIAFDADGRSCTGTLVVFLE',
 'KIIRQEEGLHAGTVLTTFTAQDPDRYMQQNIRYTKLSDPANWLKIDSVNGQITTIAVLDRESPNVKANIYNATFLASDNGIPPMSGTGTLQIYLL',
 'LRTSIAEGAPPGTSVATFSARDPDTEQLQRISYSKDYDPEDWLQVDGATGRIQTQRVLSPASPFLKDGWYRAIILALDNAIPPSTATGTLSIEIL',
 'RLAQVPEDVPLGQPLASYTAQDPDRAQQQRIKYVMGSDPAGWLAVHPENGLITAREQLDRESPFTKNSTYMAVLLAVDDGLPPATGTGTLLLTLL',
 'IVRQIKEGIPLKTPIVTVTADDPDSGLNGKVSYAISKQEPQLPQGRHFGINTETGVIHTLREIDRESIDTFRLTVVATDRAQPSERQLSTEKLVTVIVE',
 'YLARVMENTVPPNGGYVLTVNAYDADTPPLNSQVRYFLKEGDSDLFRINASSGDIALLKPLDREQQSEYTLTLVAMDTGSPPLTGTGIVRVEVQ',
 'YNVSLNETALPGTPVVTVMASDNDLGDNSKITYYLAETEHQFTVNPETGVISTTERVNCPQQTNVKSSASQKSCVFTVFARDHGSPRQDGRTYVTVNLL',
 'YSAVLSELAPTGSFVASITATDEDTGVNAQVHYDILSGNELKWFSMDPLTGLIVTTGPLDREIRDTVELSISARDGGPNPKFAYTQLKVIIL',
 'IAISFSESATSGTRLLLDAATDADVGENGVTDQYEIVAGNVDNKFRLVTTANPSGDTSYLHLETTGNLDRESRGSYQLNISARDGGSPPRFGYLQVNVTIL

In [16]:
from transformers import AutoTokenizer, EsmModel

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t36_3B_UR50D")
model = EsmModel.from_pretrained("facebook/esm2_t36_3B_UR50D")
model = model.to('cuda')

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.80it/s]
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t36_3B_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
inputs = tokenizer(seqs[0], return_tensors = 'pt').to('cuda')

In [21]:
inputs

{'input_ids': tensor([[ 0, 16, 11,  7, 10, 12, 15,  9, 17,  7, 14,  7,  6, 11, 15, 11, 12,  6,
         19, 15,  5, 19, 13, 14,  9, 11,  6,  8,  8,  8,  6, 12, 10, 19, 15, 15,
          8,  8, 13, 14,  9,  6, 22,  7, 13,  7, 13, 15, 17,  8,  6,  7, 12, 11,
         12,  4, 15, 10,  4, 13, 10,  9,  5, 10,  8,  6,  7, 19, 17, 12,  8, 12,
         12,  5,  8, 13, 15, 13,  6, 10, 11, 23, 17,  6,  7,  4,  6, 12,  7,  4,
          9,  2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [22]:
predicted_contacts = model.predict_contacts(inputs['input_ids'], inputs['attention_mask'])

In [24]:
predicted_contacts.shape

torch.Size([1, 90, 90])

In [25]:
predicted_contacts.max()

tensor(1.0000, device='cuda:0', grad_fn=<MaxBackward1>)

In [26]:
predicted_contacts.min()

tensor(1.3161e-21, device='cuda:0', grad_fn=<MinBackward1>)